In [1]:
import lkml as looker
from pprint import pprint
from google.oauth2 import service_account
import pandas_gbq
from contextlib import redirect_stdout
import snowflake.connector
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
import pandas as pd
import pandas
import os
import json
import sys
import yaml

from droughty import warehouse_target
from droughty import config

import git

pd.options.mode.chained_assignment = None

credentials = config.service_account

warehouse_name = config.warehouse_name
lookml_project = config.project_name

dimensional_inference_status = warehouse_target.dimensional_inference
sql = warehouse_target.warehouse_schema
explore_sql = warehouse_target.cube_explore_schema

if warehouse_name == 'big_query':

    # Run a Standard SQL query with the project set explicitly
    project_id = lookml_project
    df = pandas.read_gbq(sql, dialect='standard', project_id=lookml_project, credentials=credentials)

    df['description'] = df['description'].fillna('not available')

    df1 = df[['table_name','column_name','data_type','description']]

    df1['data_type'] = df1['data_type'].str.replace('TIMESTAMP','timestamp')
    df1['data_type'] = df1['data_type'].str.replace('DATE','date')
    df1['data_type'] = df1['data_type'].str.replace('INT64','number')
    df1['data_type'] = df1['data_type'].str.replace('FLOAT64','number')
    df1['data_type'] = df1['data_type'].str.replace('NUMERIC','number')
    df1['data_type'] = df1['data_type'].str.replace('STRING','string')
    df1['data_type'] = df1['data_type'].str.replace('BOOL','yesno')

    df2 = df1

    explore_df = pandas.read_gbq(explore_sql, dialect='standard', project_id=lookml_project, credentials=credentials)

    explore_df_2 = explore_df[['pk_table_name', 'pk_column_name','fk_table_name','fk_column_name','true_relationship']]

    pk_table_name_df = explore_df[['pk_table_name']]

    duplicate_explore_rows = pk_table_name_df[pk_table_name_df.duplicated(['pk_table_name'])]

    distinct_duplicate_explore_rows = duplicate_explore_rows['pk_table_name'].drop_duplicates().to_list()

    df4 = {n: grp.loc[n].to_dict('index')
        
    for n, grp in explore_df.set_index(['pk_table_name', 'pk_column_name','fk_table_name','fk_column_name','true_relationship']).groupby(level='pk_table_name')}

    d2 = df4

    def recur_dictify(frame):
        if len(frame.columns) == 1:
            if frame.values.size == 1: return frame.values[0][0]
            return frame.values.squeeze()
        grouped = frame.groupby(frame.columns[0])
        d = {k: recur_dictify(g.iloc[:,1:]) for k,g in grouped}
        return d

elif warehouse_name == 'snowflake': 

    url = URL(

        account = config.snowflake_account,
        user =  config.snowflake_user,
        schema =  config.snowflake_schema,
        database =  config.snowflake_database,
        password =  config.snowflake_password,
        warehouse= config.snowflake_warehouse,
        role =  config.snowflake_role

    )

    engine = create_engine(url)

    connection = engine.connect()

    explore_df = pd.read_sql(explore_sql, connection)

    explore_df.drop_duplicates(keep=False, inplace=True)

    explore_df['parent_table_name'] = explore_df['parent_table_name'].str.lower()
    explore_df['pk_table_name'] = explore_df['pk_table_name'].str.lower()
    explore_df['pk_column_name'] = explore_df['pk_column_name'].str.lower()
    explore_df['fk_table_name'] = explore_df['fk_table_name'].str.lower()
    explore_df['fk_column_name'] = explore_df['fk_column_name'].str.lower()

    explore_df_2 = explore_df[['parent_table_name','pk_table_name', 'pk_column_name','fk_table_name','fk_column_name','true_relationship']]

    pk_table_name_df = explore_df[['pk_table_name']]

    duplicate_explore_rows = pk_table_name_df[pk_table_name_df.duplicated(['pk_table_name'])]

    distinct_duplicate_explore_rows = duplicate_explore_rows['pk_table_name'].drop_duplicates().to_list()

    connection.close()
    engine.dispose()

    df4 = {n: grp.loc[n].to_dict('index')
        
    for n, grp in explore_df.set_index(['parent_table_name','pk_table_name', 'pk_column_name','fk_table_name','fk_column_name','true_relationship']).groupby(level='parent_table_name')}

    d2 = df4

    def recur_dictify(frame):
        if len(frame.columns) == 1:
            if frame.values.size == 1: return frame.values[0][0]
            return frame.values.squeeze()
        grouped = frame.groupby(frame.columns[0])
        d = {k: recur_dictify(g.iloc[:,1:]) for k,g in grouped}
        return d

/Users/lewischarlesbaker/.virtualenvs/droughty_dev_0.1.0/lib/python3.8/site-packages/snowflake/connector/options.py:96: UserWarning: You have an incompatible version of 'pyarrow' installed (7.0.0), please install a version that adheres to: 'pyarrow<6.1.0,>=6.0.0; extra == "pandas"'
  warn_incompatible_dep(


TypeError: not enough arguments for format string

In [2]:
explore_sql

NameError: name 'explore_sql' is not defined

In [ ]:
from pprint import pprint
from google.oauth2 import service_account
import pandas_gbq
from contextlib import redirect_stdout
import snowflake.connector
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
import pandas as pd
import pandas
import os
import json
import sys
import yaml
import git

import droughty.cube_parser.cube as cube

from droughty.cube_base_dict import d1
from droughty.cube_explore_dict import d2
from droughty.cube_explore_dict import distinct_duplicate_explore_rows
from droughty.config import schema_name

def get_all_values(nested_dictionary,explore_dictionary):
    
    for key,value in nested_dictionary.items():
    
        for explore_key, explore_value in explore_dictionary.items():

            if explore_key in key:

                explore = {


                    "cube": key,
                    "sql": "select * from"+" "+schema_name+"."+key

                }


                yield(cube.dump(explore))
        

                for key1, value1 in explore_value.items():  
                    
                    join_start = "joins: {"
                    

                    join = {


                        "joins": 
                        [
                        
                        {
                        "relationship": "belongsTo",
                        "sql": "${CUBE."+key1[0]+"}"+" = "+"${"+key1[1]+"."+key1[2]+"}",
                        "name": key1[1]

                        }
                            
                        ]

                    }
                    
                    join_end = "},"                 
                    

                    yield(join_start)

                    yield(cube.dump(join))

                    yield(join_end)

        
nested_dictionary = d1
explore_dictionary = d2

get_all_values(nested_dictionary,explore_dictionary)


for value in get_all_values(nested_dictionary,explore_dictionary):

    print(value)

In [ ]:
from droughty.cube_base_dict import d1
from droughty.cube_explore_dict import d2
from droughty.config import schema_name


In [ ]:
nested_dictionary = d1
explore_dictionary = d2

In [ ]:
d2

In [ ]:
from pprint import pprint
from google.oauth2 import service_account
import pandas_gbq
from contextlib import redirect_stdout
import snowflake.connector
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
import pandas as pd
import pandas
import os
import json
import sys
import yaml
import git

import droughty.cube_parser.cube as cube

from droughty.cube_base_dict import d1
from droughty.cube_explore_dict import d2
from droughty.cube_explore_dict import distinct_duplicate_explore_rows
from droughty.config import schema_name

def get_all_values(nested_dictionary,explore_dictionary):
    
    for key,value in nested_dictionary.items():
    
        for explore_key, explore_value in explore_dictionary.items():

            if explore_key in key:

                explore = {


                    "cube": key,
                    "sql": "select * from"+" "+schema_name+"."+key,
                    "joins ": "{"

                }


                yield(cube.dump(explore))
        

                for key1, value1 in explore_value.items(): 

                    

                    join = {


                        "joins": 
                        [
                        
                        {
                        "relationship": "belongsTo",
                        "sql": "${CUBE."+key1[0]+"}"+" = "+"${"+key1[1]+"."+key1[2]+"}",
                        "name": key1[1]

                        }
                            
                        ]

                    }
                    
                    

                    yield(cube.dump(join))


        
nested_dictionary = d1
explore_dictionary = d2

get_all_values(nested_dictionary,explore_dictionary)


for value in get_all_values(nested_dictionary,explore_dictionary):

    print(value)

In [ ]:
from pprint import pprint
from google.oauth2 import service_account
import pandas_gbq
from contextlib import redirect_stdout
import snowflake.connector
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
import pandas as pd
import pandas
import os
import json
import sys
import yaml
import git

import droughty.cube_parser.cube as cube

from droughty.cube_base_dict import d1
from droughty.cube_explore_dict import d2
from droughty.cube_explore_dict import distinct_duplicate_explore_rows
from droughty.config import schema_name

def get_all_values(nested_dictionary,explore_dictionary):
    
    for key,value in nested_dictionary.items():
    
        for explore_key, explore_value in explore_dictionary.items():

            if explore_key in key:

                explore = {


                    "cube": key,
                    "sql": "select * from"+" "+schema_name+"."+key,
                    "joins ": "{"

                }


                yield(cube.dump(explore))
        

                for key1, value1 in explore_value.items(): 

                    

                    join = {


                        "joins": 
                        [
                        
                        {
                        "relationship": key1[3],
                        "sql": "${CUBE."+key1[0]+"}"+" = "+"${"+key1[1]+"."+key1[2]+"}",
                        "name": key1[1]

                        }
                            
                        ]

                    }
                    
                    

                    yield(cube.dump(join))
                    
                    join_end = "},"                 

                yield(join_end)
                
                        
                dimension_start = "dimensions: {"

                yield(dimension_start)


                for key, value in value.items():
                    
                    if "pk" not in key[0] and "number" not in key [1]:  
                    
                        dimension = {


                            "dimension": {
                            "sql": key[0],
                            "type": key[1],
                            "name": key[0]

                            }

                        }
                    
                        yield(cube.dump(dimension))
                        
                    elif "pk" in key[0]:

                        dimension = {


                            "dimension": {
                                "primaryKey": "true",
                                "type": key[1],
                                "sql": key[0],
                                "name": key[0],
                                "description": key[2]

                            }
                        }

                        yield(cube.dump(dimension))
                    
                for key,value in nested_dictionary.items():

                    closing_syntax = "}});"

                yield (closing_syntax)


        
nested_dictionary = d1
explore_dictionary = d2

get_all_values(nested_dictionary,explore_dictionary)


for value in get_all_values(nested_dictionary,explore_dictionary):

    print(value)

In [ ]:
d2

In [ ]:
query = (
    "SELECT name FROM `bigquery-public-data.usa_names.usa_1910_2013` "
    'WHERE state = "TX" '
    "LIMIT 100"
)
query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
)  # API request - starts the query

results = query_job.result()  # Wait for query to complete.
print("Got {} rows.".format(results.total_rows))

In [ ]:
query_job = client.query(query, dialect='standard', project_id=lookml_project, credentials=credentials)
client = bigquery.Client() 
    
results = query_job.result()  # Wait for query to complete.

print("Got {} rows.".format(results.total_rows))

In [ ]:
from google.cloud import bigquery
client = bigquery.Client()

query = (
    "SELECT name FROM `bigquery-public-data.usa_names.usa_1910_2013` "
    'WHERE state = "TX" '
    "LIMIT 100"
)
query_job = client.query(query, dialect='standard', project_id=lookml_project, credentials=credentials)

results = query_job.result()  # Wait for query to complete.
print("Got {} rows.".format(results.total_rows))

In [3]:
import lkml as looker
from pprint import pprint
from google.oauth2 import service_account
import pandas_gbq
from contextlib import redirect_stdout
import snowflake.connector
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
import pandas as pd
import pandas
import os
import json
import sys
import yaml

from droughty import warehouse_target
from droughty import config

import git

pd.options.mode.chained_assignment = None

credentials = config.service_account

warehouse_name = config.warehouse_name
lookml_project = config.project_name

dimensional_inference_status = warehouse_target.dimensional_inference
sql = warehouse_target.warehouse_schema
explore_sql = warehouse_target.cube_explore_schema

TypeError: not enough arguments for format string